In [2]:
import torch

In [3]:
a = torch.tensor([[1,2,3],[4,5,6],[7,8,9]])

In [4]:
print(a)

tensor([[1, 2, 3],
        [4, 5, 6],
        [7, 8, 9]])


In [13]:
print(a[1][1:])
print(a[2][1:])
b = torch.zeros(2,2)
print(b)

tensor([5, 6])
tensor([8, 9])
tensor([[0., 0.],
        [0., 0.]])


In [14]:
b[0] = a[1][1:]
b[1] = a[2][1:]
print(b)

tensor([[5., 6.],
        [8., 9.]])


In [3]:
c = torch.zeros(10,10)
for i in range(10):
    for j in range(10):
        c[i][j] = i * 10 + j

In [4]:
print(c)

tensor([[ 0.,  1.,  2.,  3.,  4.,  5.,  6.,  7.,  8.,  9.],
        [10., 11., 12., 13., 14., 15., 16., 17., 18., 19.],
        [20., 21., 22., 23., 24., 25., 26., 27., 28., 29.],
        [30., 31., 32., 33., 34., 35., 36., 37., 38., 39.],
        [40., 41., 42., 43., 44., 45., 46., 47., 48., 49.],
        [50., 51., 52., 53., 54., 55., 56., 57., 58., 59.],
        [60., 61., 62., 63., 64., 65., 66., 67., 68., 69.],
        [70., 71., 72., 73., 74., 75., 76., 77., 78., 79.],
        [80., 81., 82., 83., 84., 85., 86., 87., 88., 89.],
        [90., 91., 92., 93., 94., 95., 96., 97., 98., 99.]])


In [7]:
m5 = torch.zeros(8,8)
num_nodes = 8
for p in range(num_nodes):
    m5[p] = c[-num_nodes + p][-num_nodes:]
print(m5)

tensor([[22., 23., 24., 25., 26., 27., 28., 29.],
        [32., 33., 34., 35., 36., 37., 38., 39.],
        [42., 43., 44., 45., 46., 47., 48., 49.],
        [52., 53., 54., 55., 56., 57., 58., 59.],
        [62., 63., 64., 65., 66., 67., 68., 69.],
        [72., 73., 74., 75., 76., 77., 78., 79.],
        [82., 83., 84., 85., 86., 87., 88., 89.],
        [92., 93., 94., 95., 96., 97., 98., 99.]])


In [8]:
print(c[-8][-8:])


tensor([22., 23., 24., 25., 26., 27., 28., 29.])
